# *Gaia*-*Kepler* cross-match

Using [Kepler Gaia Fun!](https://gaia-kepler.fun/) I will cross-match the Serenelli sample with Gaia.

In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table

In [2]:
kepler_gaia_path = '/Users/ajl573/Data/kepler_dr2_4arcsec.fits'

with fits.open(kepler_gaia_path) as hdul:
    data = hdul[1].data

In [3]:
data.columns

ColDefs(
    name = 'solution_id'; format = 'K'; null = 999999
    name = 'designation'; format = '28A'
    name = 'source_id'; format = 'K'; null = 999999
    name = 'random_index'; format = 'K'; null = 999999
    name = 'gaia_ref_epoch'; format = 'D'
    name = 'ra'; format = 'D'; unit = 'deg'
    name = 'ra_error'; format = 'D'; unit = '10**-3 arcsec'
    name = 'dec'; format = 'D'; unit = 'deg'
    name = 'dec_error'; format = 'D'; unit = '10**-3 arcsec'
    name = 'parallax'; format = 'D'; unit = '10**-3 arcsec'
    name = 'parallax_error'; format = 'D'; unit = '10**-3 arcsec'
    name = 'parallax_over_error'; format = 'E'
    name = 'pmra'; format = 'D'; unit = '10**-3 arcsec yr-1'
    name = 'pmra_error'; format = 'D'; unit = '10**-3 arcsec yr-1'
    name = 'pmdec'; format = 'D'; unit = '10**-3 arcsec yr-1'
    name = 'pmdec_error'; format = 'D'; unit = '10**-3 arcsec yr-1'
    name = 'ra_dec_corr'; format = 'E'
    name = 'ra_parallax_corr'; format = 'E'
    name = 'ra_pmra_cor

In [4]:
df_kg = Table(data).to_pandas()
df_kg.head()

,solution_id,designation,source_id,random_index,gaia_ref_epoch,ra,ra_error,dec,dec_error,parallax,...,mass_err2,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_ang_dist
0,1635721458409799680,Gaia DR2 2050233807328471424,2050233807328471424,689405501,2015.5,291.038681,0.035878,36.598031,0.047832,1.493916,...,-0.475,DSEP,0,0,0,10.126,9.667,9.559,none,0.156160
1,1635721458409799680,Gaia DR2 2050233601176543104,2050233601176543104,447382505,2015.5,291.043057,0.019919,36.593767,0.027553,2.678194,...,-0.233,DSEP,0,0,1,11.656,11.193,11.094,none,0.128262
2,1635721458409799680,Gaia DR2 2050230543159814656,2050230543159814656,1004823384,2015.5,291.055892,0.019394,36.559886,0.026621,1.723413,...,-0.947,DSEP,0,0,0,7.467,6.853,6.722,none,0.146787
3,1635721458409799680,Gaia DR2 2050230611879323904,2050230611879323904,1253666653,2015.5,291.095373,0.016886,36.564903,0.022993,1.183652,...,-0.157,DSEP,0,0,0,10.852,10.662,10.627,none,0.116420
4,1635721458409799680,Gaia DR2 2050231848829944320,2050231848829944320,298032508,2015.5,291.137620,0.019560,36.577344,0.025196,1.168913,...,-0.040,DSEP,1,1,1,13.665,13.262,13.118,conf,0.114502


Flag nearest neighbours

In [5]:
def nearest_neighbours(df, source_id, ang_dist):
    ''' Returns a nearest neighbour flag for each source id '''
    sort = df.sort_values(ang_dist)  # Sort by angular distance
    sort['flag'] = 1  # Flag all as neighbour
    sort.loc[sort[source_id].duplicated(keep='first'), 'flag'] = 0  # Unflag duplicates
    return sort['flag'].sort_index()

In [6]:
df_kg['nearest_neighbour'] = nearest_neighbours(df_kg, 'kepid', 'kepler_gaia_ang_dist')
df_kg.head()

,solution_id,designation,source_id,random_index,gaia_ref_epoch,ra,ra_error,dec,dec_error,parallax,...,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_ang_dist,nearest_neighbour
0,1635721458409799680,Gaia DR2 2050233807328471424,2050233807328471424,689405501,2015.5,291.038681,0.035878,36.598031,0.047832,1.493916,...,DSEP,0,0,0,10.126,9.667,9.559,none,0.156160,1
1,1635721458409799680,Gaia DR2 2050233601176543104,2050233601176543104,447382505,2015.5,291.043057,0.019919,36.593767,0.027553,2.678194,...,DSEP,0,0,1,11.656,11.193,11.094,none,0.128262,1
2,1635721458409799680,Gaia DR2 2050230543159814656,2050230543159814656,1004823384,2015.5,291.055892,0.019394,36.559886,0.026621,1.723413,...,DSEP,0,0,0,7.467,6.853,6.722,none,0.146787,1
3,1635721458409799680,Gaia DR2 2050230611879323904,2050230611879323904,1253666653,2015.5,291.095373,0.016886,36.564903,0.022993,1.183652,...,DSEP,0,0,0,10.852,10.662,10.627,none,0.116420,1
4,1635721458409799680,Gaia DR2 2050231848829944320,2050231848829944320,298032508,2015.5,291.137620,0.019560,36.577344,0.025196,1.168913,...,DSEP,1,1,1,13.665,13.262,13.118,conf,0.114502,1


In [7]:
serenelli_path = '../data/serenelli2017/s17_seis.csv'

df_s17 = pd.read_csv(serenelli_path)
df_s17.head()

,KIC,numax,enumax,Dnu,eDnu,Length,HBR
0,1435467,1382.311,19.038,70.558,0.087,938.1,1.662
1,2010607,674.919,146.523,42.479,2.185,28.9,1.547
2,2309595,643.208,11.226,39.029,0.721,28.9,1.767
3,2450729,1053.105,114.904,61.910,2.539,28.9,1.470
4,2837475,1629.761,18.396,75.720,0.129,1029.4,1.767


In [11]:
df_kg.rename(columns={'kepid': 'KIC'}, inplace=True)

In [18]:
df = pd.merge(df_s17['KIC'], df_kg, on='KIC')
print('Length =', len(df))
df.head()

Length = 453


,KIC,solution_id,designation,source_id,random_index,gaia_ref_epoch,ra,ra_error,dec,dec_error,...,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_ang_dist,nearest_neighbour
0,1435467,1635721458409799680,Gaia DR2 2051714265379610624,2051714265379610624,1642703359,2015.5,292.082639,0.022439,37.059874,0.026381,...,DSEP,0,0,0,7.983,7.753,7.718,none,0.076482,1
1,2010607,1635721458409799680,Gaia DR2 2051083248784888576,2051083248784888576,951146959,2015.5,290.505591,0.020091,37.458935,0.026062,...,DSEP,0,1,1,10.427,10.204,10.155,cand,0.068342,1
2,2309595,1635721458409799680,Gaia DR2 2051847684243759104,2051847684243759104,789196893,2015.5,292.901449,0.019937,37.624510,0.027366,...,DSEP,0,0,1,10.040,9.604,9.500,none,0.032582,1
3,2450729,1635721458409799680,Gaia DR2 2051667914096024448,2051667914096024448,1066183801,2015.5,293.073631,0.021554,37.716582,0.026427,...,DSEP,0,0,0,9.585,9.333,9.283,none,0.031511,1
4,2837475,1635721458409799680,Gaia DR2 2099526837747055232,2099526837747055232,1298060034,2015.5,287.548574,0.023019,38.082134,0.027463,...,DSEP,0,0,0,7.649,7.481,7.464,none,0.122110,1


In [19]:
df.to_csv('../data/kepler_gaia/s17_gaia_4arcsec.csv', index=False)